In [32]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import numpy as np
import pandas as pd
import time
import re

In [2]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [33]:
class utilities:
    
    #def 
    
    def wait(self, seconds):

        s = np.random.normal(0, 1)
        time.sleep(seconds*abs(s))
    
    def get_element_by_path(self, driver, xpath):
        try:
            return driver.find_element_by_xpath(xpath)
        except Exception as e:
            print(e)
            return None

In [34]:
class IndeedItemParser:
    def __init__(self):
        self.driverPath = "C:\\Users\\User\\Documents\\selenium\\driver\\chromedriver.exe"
        self.utilities = utilities();
        
    def _get_title(self, driver):
        try:
            title = driver.find_element_by_xpath("//*[@class='jobsearch-DesktopStickyContainer']//h3")
        except Exception as e:
            print(e)
        return title.text
    
    def _get_name(self, driver):
        
        try:    
            name = driver.find_element_by_xpath("//*[contains(@class,'jobsearch-InlineCompanyRating')]//div[1]")
            return name.text
        except Exception as e:
            print(e)
        
    
    def _get_address(self,driver):
         try:   
            address = driver.find_element_by_xpath("//*[contains(@class,'jobsearch-InlineCompanyRating')]//div[3]")
            if address.text == "-":
                address = driver.find_element_by_xpath("//*[contains(@class,'jobsearch-InlineCompanyRating')]//div[4]")
            return address.text
         except Exception as e:
            address = driver.find_element_by_xpath("//span[@class='jobsearch-JobMetadataHeader-iconLabel'][1]")
            return address.text
    
    def _get_salaire(self,driver, description):
        #jobsearch-JobMetadataHeader-item 
        """
        pattern = r'[[S|s]alaire?[\s+]?:?[\s+]?(.*)?[\s+]?|(.*)?[\s+]?(par|\/)?[\s+]?(an|mois|jour|heure)'
        text = description
        try:
            text = driver.find_element_by_xpath("//*[contains(@class,'jobsearch-JobMetadataHeader-item')]//span[1]").text
            print("first :", text)
            if text == "":
                raise Exception("empty")
        except Exception as e:
            print(e)
            try:   
                text = driver.find_element_by_xpath("//*[contains(@class,'icl-IconFunctional--salary')]//*[contains(@class,'jobsearch-JobMetadataHeader-iconLabel')]").text
                print("second : ",text)
            except Exception as e:
                print(e)
                print("troisième")
                
        outer = re.compile(pattern)
        m = outer.search(text)
        if m is not None:
              return m.group(1)
        return "-"
        """
        return "-"
            
    
    def _get_description(self,driver):
        try:
            #jobDescriptionText
            e_description = driver.find_element_by_id("jobDescriptionText")
            return e_description.text
        except Exception as e:
            print(e)
    
    def _get_date(self,driver,url,name):
       # print(url)
        try:
            date_str = driver.find_element_by_xpath("//*[@class='jobsearch-JobMetadataFooter']")
            date_str_full = date_str.text
            date_tbl = date_str_full.split(" ")
            count_str = date_tbl[4]
            label = date_tbl[5]
            
            if name in date_str_full:
                date_str_full = date_str_full.replace(name, "")
                date_tbl = date_str_full.split(" ")
                count_str = date_tbl[5]
                label = date_tbl[6]
                
            if count_str == "a" :
                count_str = date_tbl[5]
                label = date_tbl[6]
                
            #print("date_str", date_tbl)
            date = datetime.now()
            
            if count_str == "30+":
                return date - timedelta(days=30)
            
            count = int(count_str)
            if "jour" in label:
                date = date - timedelta(days=count)
            elif "heur" in label:
                date = date - timedelta(hours=count)
            return date;
        except Exception as e:
            print(e)
    
    
    def parse(self,url):
        driver = webdriver.Chrome(self.driverPath)
        driver.get(url)
        driver.maximize_window()
        
        source = driver.page_source
        title = self._get_title(driver)
        name = self._get_name(driver)
        address = self._get_address(driver)
        date = self._get_date(driver, url,name)
        description = self._get_description(driver)
        salaire = self._get_salaire(driver,description)
        
        driver.quit()
        
        return title, name, address, date, salaire, description, source

In [395]:
#item_parser = IndeedItemParser()
#title, name, address, date, salaire,description,source = item_parser.parse("https://www.indeed.fr/voir-emploi?cmp=ECO--SI&t=Technicien+Support+Informatique&jk=dc1a032be2312c84&sjdu=QwrRXKrqZ3CNX5W-O9jEvX5ViC7ZmtCbUlpLMENSqzeEixTsazLVFu1bx1PMxHpgK0V934oVal0W7yYsbyobejEeYX4jvfhEEiUY8zoCrBk&tk=1dm9ttu0492gm801&adid=314524947&vjs=3")
#print(source)

In [35]:
import os.path
from os import path
import json

class IndeedParserStateObject(object):
    def __init__(self, data):
        self.__dict__ = json.loads(data)
        
class IndeedParserStateHandler:
    
    def __init__(self):
        self.filename = "indeed.parser.state.json"
        self.object = self._load_file()
        #self.object = IndeedParserStateObject(self.data)
    
    def save_state(self, job, location, page_request="", page_item_index=-1):
        self.object["job"] = job
        self.object["location"] = location
        self.object["page_request"] = page_request
        self.object["page_item_index"]  = page_item_index
        #self.data  = json.dumps(self.object)
        
        #data = json.dumps(self.object)
        self._save_file(self.object)
        #self.object = IndeedParserStateObject(self.data)
        
    def is_current_job(self, job):
        if (self.object["job"] == "") | (self.object["job"] == job):
            return True
        return False
    
    def is_current_location(self, location):
        if (self.object["location"] == "") | (self.object["location"] == location):
            return True
        return False
    
    def is_current_page_request(self, page_request):
        if (self.object["page_request"] == "") | (self.object["page_request"] == page_request):
            return True
        return False
    
    def _save_file(self, data):
         with open(self.filename, 'w') as outfile:
                json.dump(data, outfile)
                
    def _load_file(self):
        
        if path.exists(self.filename) == True:
            with open(self.filename) as json_file:
                data = json.load(json_file)
        else:
            data = {
                     "job": "",
                     "location":"",
                     "page_request":"",
                     "page_item_index":-1,
                    }
            with open(self.filename, 'w') as outfile:
                json.dump(data, outfile)
        
        return data


In [41]:
from datetime import datetime, timedelta
import os.path
import os
class IndeedPaser:
    def __init__(self):
        self.website = "https://www.indeed.fr"
        self.driverPath = "C:\\Users\\User\\Documents\\selenium\\driver\\chromedriver.exe"
        #self.dataset = pd.DataFrame(columns=['URL', 'Titre','Nom entreprise','Adresse','Date de publication', 'description'])
        
        self.dataset = pd.read_csv("indeed.csv")
        #if os.path.exists('indeed.csv') == True:
        #    self.dataset = pd.read_csv("indeed.csv")
        #else:
            #URL,Titre,Nom entreprise,Adresse,Date de publication,salaire, description
        #    self.dataset = pd.DataFrame(columns=['URL','Titre','Nom entreprise','Adresse','Date de publication','salaire','niveau experience', 'description'])
        
        self.jobs = ["data scientist", "développeur", "data analyst", "business intelligence"]
        self.locations = ["Paris","Lyon", "Toulouse", "Nantes", "Bordeaux"]
        #self.locations = ["Paris", "Lyon", "Toulouse", "Nantes", "Bordeaux"]
        self.utilities =  utilities()
        self.indeed_item_parser = IndeedItemParser()
        self.parser_state = IndeedParserStateHandler()
    
    def _get_pages_counts(self,driver):
        #searchCountPages
        searchCountPages_elt = driver.find_element_by_id("searchCountPages")
        searchCountPages = searchCountPages_elt.text.split()
        if len(searchCountPages) == 6:
            searchCountPages = int("{0}{1}".format(searchCountPages[3],searchCountPages[4])) 
        else :
            searchCountPages = searchCountPages[3]  
        
        return (int(searchCountPages) // 18)
    
    def parse(self):
        browser = webdriver.Chrome(self.driverPath)
        browser.get(self.website)
        browser.maximize_window()
        
        for job in self.jobs:
            
            #if self.parser_state.is_current_job(job) == False:
            #    continue
                    
            for location in self.locations:
                
                #if self.parser_state.is_current_location(location) == False:
                #    continue
                #self.parser_state.save_state(job, location)
                
                query = "https://www.indeed.fr/jobs?q={0}&l={1}".format(job, location)
                browser.get(query)
                    
                pages_count = self._get_pages_counts(browser)
                
                for page_index in range(1, pages_count):
                    full_query = "{0}&start={1}".format(query,page_index)
                    
                    browser.get(full_query)
                    
                    items = browser.find_elements_by_xpath("//*[contains(@class,'clickcard')]")
                    dataset_len = len(self.dataset)
                    
                    for index_i,item in enumerate(items): 
                        title = item.find_element_by_xpath(".//*[contains(@class,'jobtitle')]")
                        item_link = title.get_attribute("href")
                        
                        try:
                            if (self.dataset["URL"].isin([item_link]).any()):
                                print("aready parsed, skip", item_link)
                            else:
                                
                                title, name, address, date,salaire, description, source = self.indeed_item_parser.parse(item_link)
                                
                                if (self.dataset["Titre"].isin([title]).any()
                                 &
                                self.dataset["Nom entreprise"].isin([name]).any()
                                 & 
                                self.dataset["Adresse"].isin([address]).any()
                                 &
                                self.dataset["description"].isin([description]).any()):
                                    print("doublon, skip.")
                                    continue
                                        
                                
                                direct_cols = [item_link, title, name, address, date, salaire, description]
                                remain_cols = ['' for item in range((len(self.dataset.columns) - len(direct_cols)))] 
                                all_cols = direct_cols + remain_cols 
                                self.dataset.loc[dataset_len + index_i] = all_cols
                                
                                #archivage des page html
                                source_file_name = "source_file.{0}.html".format(dataset_len + index_i)
                                if not os.path.exists(source_file_name):
                                    file = open("pages/{0}".format(source_file_name), "w")
                                    source = item_link + "----------------" + source 
                                    file.write(source) 
                                    file.close() 
                                
                                print("len(dataset)", len(self.dataset))
                                
                        except Exception as e:
                            print(e)
                    
                    duplicates_cols = ["Titre", "Nom entreprise", "Adresse", "description"]
                    self.dataset.drop_duplicates(subset = duplicates_cols, keep = False, inplace = True)
                    self.dataset.to_csv("indeed.csv",index=False)
                    print("indeed.csv saved")
                    self.dataset = pd.read_csv("indeed.csv")
                
              

In [ ]:
test = IndeedPaser()
test.parse()

len(dataset) 716
len(dataset) 717
len(dataset) 718
len(dataset) 719
len(dataset) 720
len(dataset) 721
aready parsed, skip https://www.indeed.fr/rc/clk?jk=cdf53cf5e71884f6&fccid=feb484a8aef310be&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=ac50f6be14a634d7&fccid=17f4236a40d7d727&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=a4eaba2c27cbc6d5&fccid=ec8a0bd60be9a6f5&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=ef40d1a5f0e1e2fe&fccid=08cedfdd897011ae&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=88c6253bbc63fc4e&fccid=028bbd854233c1f1&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=d3859b022ec6e5ad&fccid=df6948c9b8da6236&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=df2af7d436d23b5d&fccid=e5e041b12c825086&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=715ef6bee0dc4d80&fccid=d51961f0c6d6c8df&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=b4ffdd1614d1e3ae&fccid=2301bee5373a0b4a&vjs=3
aready parsed, s

doublon, skip.
doublon, skip.
doublon, skip.
indeed.csv saved
doublon, skip.
doublon, skip.
doublon, skip.
doublon, skip.
doublon, skip.
aready parsed, skip https://www.indeed.fr/rc/clk?jk=ac50f6be14a634d7&fccid=17f4236a40d7d727&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=88c6253bbc63fc4e&fccid=028bbd854233c1f1&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=cdf53cf5e71884f6&fccid=feb484a8aef310be&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=d3859b022ec6e5ad&fccid=df6948c9b8da6236&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=0afed7661a2a5af6&fccid=7077ed64aecd53a3&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=a4eaba2c27cbc6d5&fccid=ec8a0bd60be9a6f5&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=b4ffdd1614d1e3ae&fccid=2301bee5373a0b4a&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=df2af7d436d23b5d&fccid=e5e041b12c825086&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=ef40d1a5f0e1e2fe&fccid=08ce

doublon, skip.
doublon, skip.
doublon, skip.
doublon, skip.
aready parsed, skip https://www.indeed.fr/rc/clk?jk=715ef6bee0dc4d80&fccid=d51961f0c6d6c8df&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=4ec9be2493215b5a&fccid=f15e5e4a46544874&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=aa7035f857a1e340&fccid=ef1f17d79d6494fe&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=d3859b022ec6e5ad&fccid=df6948c9b8da6236&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=62422466416772ba&fccid=58db872dee6c6b74&vjs=3
aready parsed, skip https://www.indeed.fr/company/Four-Elements-Capital/jobs/Financial-Data-Scientist-c8588be471a67852?fccid=a8629176f3a10d62&vjs=3
aready parsed, skip https://www.indeed.fr/company/ACTIVUS-Group/jobs/Data-Scientist-053959c5fc230c71?fccid=e7a47c5f5575c789&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=90a235627582bc48&fccid=2b596b7e420f89d3&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=69e4fcadd6e8acce&fccid

doublon, skip.
doublon, skip.
doublon, skip.
indeed.csv saved
doublon, skip.
doublon, skip.
aready parsed, skip https://www.indeed.fr/rc/clk?jk=1ea91098ccbca193&fccid=df6948c9b8da6236&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=e131356a90e02c3a&fccid=518f77ef3e7cf0da&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=ca631bbf666bdb38&fccid=81fd9b0296f772b6&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=d3f21261a509508d&fccid=9b6c70e895ae496c&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=c550e9e45ecec7c4&fccid=ff4d5564b774d59b&vjs=3
aready parsed, skip https://www.indeed.fr/company/AID-add-intelligence-to-Data/jobs/Data-Scientist-fc87a2f33bb23599?fccid=22db5fb4d6f285e2&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=1bfc0b2c6443fe0d&fccid=9b246b5aaacc1ef2&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=3c3715a74a7ebf22&fccid=9b246b5aaacc1ef2&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=806d52238c7b9bd4&fccid=0a

doublon, skip.
doublon, skip.
doublon, skip.
indeed.csv saved
doublon, skip.
doublon, skip.
aready parsed, skip https://www.indeed.fr/rc/clk?jk=e131356a90e02c3a&fccid=518f77ef3e7cf0da&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=ca631bbf666bdb38&fccid=81fd9b0296f772b6&vjs=3
aready parsed, skip https://www.indeed.fr/company/AID-add-intelligence-to-Data/jobs/Data-Scientist-fc87a2f33bb23599?fccid=22db5fb4d6f285e2&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=c9e6e768c2d2fb6a&fccid=ba550cb3a1a4f7de&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=1ea91098ccbca193&fccid=df6948c9b8da6236&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=c550e9e45ecec7c4&fccid=ff4d5564b774d59b&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=d3f21261a509508d&fccid=9b6c70e895ae496c&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=1bfc0b2c6443fe0d&fccid=9b246b5aaacc1ef2&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=9d336a7f7601a88e&fccid=8a

...

doublon, skip.
doublon, skip.
doublon, skip.
doublon, skip.
doublon, skip.
indeed.csv saved
doublon, skip.
aready parsed, skip https://www.indeed.fr/company/Sports-Decisions/jobs/Stage-497fc763aea937b2?fccid=6671a7d4786e7b35&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=a57a15d1bbf51db6&fccid=b93d983fad30b3d7&vjs=3
aready parsed, skip https://www.indeed.fr/company/SmartKeyword/jobs/D%C3%A9veloppeur-Full-Stack-b2c034aa6e784f68?fccid=46e584d0edb13d99&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=fb6b976c2106d58b&fccid=e72e741a3b75e025&vjs=3
aready parsed, skip https://www.indeed.fr/company/Data-Impact-Analytics/jobs/D%C3%A9veloppeur-Python-Junior-1a815b2313a4f6fc?fccid=f117fa20a31c19b8&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=2637736e435052a4&fccid=935c7a0afc5691c7&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=a126d3cb85efce36&fccid=9ff6bf8aacf150d4&vjs=3
aready parsed, skip https://www.indeed.fr/company/AXIALEASE/jobs/D%C3%A9veloppeur

doublon, skip.
doublon, skip.
doublon, skip.
doublon, skip.
doublon, skip.
indeed.csv saved
doublon, skip.
doublon, skip.
aready parsed, skip https://www.indeed.fr/rc/clk?jk=39d7b2af453ea6df&fccid=c3f5b5410a0388af&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=090a8d09cfb70dbf&fccid=0bc9d524dbf6f889&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=c530e397c6642334&fccid=14b7c83f2c02772e&vjs=3
aready parsed, skip https://www.indeed.fr/company/Manadge/jobs/D%C3%A9veloppeur-Back-End-fdb26ed5281a3f84?fccid=9881f9de9a558a63&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=a05d78b661afd8f7&fccid=5846066f6d17e6dd&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=dabac52407fe85b5&fccid=df6948c9b8da6236&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=41ee5d36cc760326&fccid=dd616958bd9ddc12&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=6821f4425d77f5e3&fccid=63f02c7e31cdfb5d&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=748f2

doublon, skip.
doublon, skip.
doublon, skip.
doublon, skip.
doublon, skip.
indeed.csv saved
'charmap' codec can't encode character '\u0300' in position 37328: character maps to <undefined>
aready parsed, skip https://www.indeed.fr/rc/clk?jk=27bfe10e9eccd242&fccid=7ea84b0d6f55d1cc&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=4578606fe97d251b&fccid=b920aab5fd60595d&vjs=3
aready parsed, skip https://www.indeed.fr/company/Cabinet-Hezard-&-Associ%C3%A9s/jobs/D%C3%A9veloppeur-Logiciels-Net-62d4605d373b4d3e?fccid=7392677ab76287d5&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=b4fdb3d10db75803&fccid=cac2efee9fb70684&vjs=3
aready parsed, skip https://www.indeed.fr/company/Team'Doc/jobs/D%C3%A9veloppeur-Ios-Swift-Junior-24d260852ddbb9cf?fccid=da139762215256e1&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=10bcd59433b0aa85&fccid=b3fe6b9a3cf9f1d4&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=dcac6a5c06414a72&fccid=30ccadd026903dc7&vjs=3
aready parsed,

aready parsed, skip https://www.indeed.fr/rc/clk?jk=c802d29a8072a827&fccid=908941108e74fc9c&vjs=3
doublon, skip.
doublon, skip.
doublon, skip.
doublon, skip.
indeed.csv saved
doublon, skip.
doublon, skip.
doublon, skip.
aready parsed, skip https://www.indeed.fr/rc/clk?jk=8b4cc1715c6c1336&fccid=56c579d1ed9f7bed&vjs=3
aready parsed, skip https://www.indeed.fr/company/OCTOPUS-IT/jobs/D%C3%A9veloppeur-Fullstack-React-Node-Js-a6f40b2fe4d35968?fccid=7358eb8948faec3a&vjs=3
aready parsed, skip https://www.indeed.fr/company/Cabinet-Hezard-&-Associ%C3%A9s/jobs/D%C3%A9veloppeur-Logiciels-Net-62d4605d373b4d3e?fccid=7392677ab76287d5&vjs=3
aready parsed, skip https://www.indeed.fr/company/MyBestPro/jobs/D%C3%A9veloppeur-PHP-Symfony-83e2ebfbb258ec40?fccid=a91fb0eff298a9c2&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=4578606fe97d251b&fccid=b920aab5fd60595d&vjs=3
aready parsed, skip https://www.indeed.fr/company/TSI/jobs/D%C3%A9veloppeur-Back-End-6036f156040fe6fd?fccid=b4b964f1fed3a869&vjs

doublon, skip.
doublon, skip.
doublon, skip.
doublon, skip.
indeed.csv saved
doublon, skip.
doublon, skip.
aready parsed, skip https://www.indeed.fr/rc/clk?jk=ceb6329904843b30&fccid=6f4826f583f9897c&vjs=3
aready parsed, skip https://www.indeed.fr/company/Cabinet-Hezard-&-Associ%C3%A9s/jobs/D%C3%A9veloppeur-Java-Full-Stack-7477e0a7a394cfee?fccid=7392677ab76287d5&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=748f2c1c8240a0b6&fccid=de3433c9c01176f7&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=87fd6dce58955258&fccid=aefbced665b7b78f&vjs=3
aready parsed, skip https://www.indeed.fr/company/CACIIS/jobs/D%C3%A9veloppeur-Python-3c4ee40bef6c511b?fccid=f70036a8c6ec7866&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=3d330c9d7ddc0f7f&fccid=fe72b8eda90faf79&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=6821f4425d77f5e3&fccid=63f02c7e31cdfb5d&vjs=3
aready parsed, skip https://www.indeed.fr/rc/clk?jk=0da82535b49a23ec&fccid=4e85460ad667e517&vjs=3
aready p

In [14]:
len(test.dataset)

3632

In [10]:
pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install django

Note: you may need to restart the kernel to use updated packages.


In [403]:
class CsvToMongodbMigratorTool:
    
    def __init__(self):
        self.dataset = pd.read_csv("indeed.csv")
        self.mongodbdao = IndeedMongodbDao()
        
    def migrate(self):
        records = json.loads(self.dataset.T.to_json()).values()
        self.mongodbdao.insert_data_bulk(records)
        

In [82]:
#migratorTool = CsvToMongodbMigratorTool()
#migratorTool.migrate()

In [407]:
from pymongo import MongoClient 
from pymongo import errors 
from django.core.validators import URLValidator
from django.core.exceptions import ValidationError
class IndeedMongodbDao:
    def __init__(self):
        self.conn = MongoClient() 
        self.db = self.conn.Indeed
        self.collection = self.db.data
        self._create_index()
    
    def _create_index(self):
        index_name = 'url'
        if index_name not in self.collection.index_information():
            self.collection.create_index(index_name, unique=True)
    
    def _valid_url_format(self,url):
        val = URLValidator()
        try:
            val(url)
        except ValidationError as e:
            raise Exception('bad format for url {}'.format(ur))
    
    def insert_data_bulk(self,data):
        self.collection.insert_many(data)
    
    def insert_data(self, url, title, name, address, publication_date, description):
        
        try:
            if url == "":
                raise Exception('url cannot be empty {}'.format(ur))

            self._valid_url_format(url)

            if title == "":
                raise Exception('title cannot be empty {}'.format(title))

            if name == "":
                raise Exception('the name of company cannot be be empty {}'.format(title))

            if description == "":
                raise Exception('description of company cannot be be empty {}'.format(title))


            line_to_insert = {
                                "URL": url,
                                "title":title,
                                "name":name,
                                "adresse":address,
                                "publication_date":publication_date,
                                "description":description
                             }

            # Insert Data 
            result = self.collection.insert_one(line_to_insert) 
        except errors.DuplicateKeyError as e:
            print('DuplicateKeyError : url {0} aready exist'.format(url))
    
    def get_all_data(self):
        data = self.collection.find({})
        return data
    
    

In [128]:
import numpy as np 
class IndedRegexParser:
    
    def __init__(self, descriptif_text ):
        self.descriptif_text = descriptif_text
    
    def _get_items(self,pattern):   
        result_re = re.findall(pattern,self.descriptif_text)
        result = []
        for item_l_1 in result_re:
            for item_l_2 in item_l_1:
                if item_l_2 != "":
                    result.append(item_l_2)
        return result
    
    def _get_niveau_experience(self):
        #Niveau d'expérience : regex true
        result = self._get_items('([(e|E)\w+]xpérience?:?\s\d{1,15} ans?)|([(e|E)\w+]xpérience?\s?de\s?\d{1,15} ans?)|(\d{1,15} ans d\'[e|E]xpérience[s]?)')
        if len(result) == 0:
            outer = re.compile(r'Expérience:\s*.+:( (\w) (an|ans) \(((R|r)equis|(s|S)ouhaité)\))')
            m = outer.search(self.descriptif_text)
            if m is not None:
                return m.group(1)
            #return []
    
    def _get_salaires(self):
        #Salaire ajusté en fonction du coût de la vie dans les différentes villes : regex true
        #return self._get_items('(\d{2,3}\s?(k|K)?)|(\d{2,3}\s?\d{3,5}\s?(k|K)?)|([(s|S)]alaire?[\s]?:?[\s]?\d{2,6}k?)|([(r|R)]émunération?[\s]?:?[\s]?\d{2,6}k?)|([(g|G)]ratification?[\s]?:?[\s]?\d{2,6}k?)') 
        #[[S|s]alaire?[\s+]?:?[\s+]?(.*)e?[\s+]?\/an
        outer = re.compile(r'[[S|s]alaire?[\s+]?:?[\s+]?(.*)e?[\s+]?\/(an|mois)|(.*)?[\s+]?par?[\s+]?mois')
        m = outer.search(self.descriptif_text)
        if m is not None:
            return m.group(1)
        return []
            
        
    
    def get_langages_prog(self):
        #Langages de programmation requis (R, Python, Excel (VBA), SQL, C++, Java, SAS...): regex true
        return self._get_items('(R+[^\w])|([(p|P)\w+]ython?)|(Excel?)|(VBA?)|(C\+\+?)||(C?)|(Asp.net?)|(SQL?)|(NoSQL?)|(Linux?)|(MySQL?)|(MongoDB?)|(DBMaria?)|(SQL Server?)||(Java?)|(JavaScript?)|(PHP?\s?\d{1,}?)|(HTML?\s?\d{1,}?)|(CSS?\s?\d{1,}?)|(SAS?)|(C#?)|(Ruby?)|(Swift?)|(Objective-C?)|(VB.NET?)|(Kotlin?)|(Scala?)|(Bash?)|(PowerShell?)|(Shell?)|(Front End?)|(Back End?)')
    
    def get_outils_re(self):
        #Outils requis (Tableau, PowerBI...) : regex true
        return self._get_items('([(t|T)\w+]ableau?)|([(p|P)\w+]ower[(b|B)\w+][(i|I)\w+]?)|([(s|S)\w+]ymfony?\s?\d{1,}?)|([(j|J)\w+][(q|Q)\w+]uery?\s?\d{1,}?)|([(a|A)\w+]ngular?[\s?\d{1,}]?)|([(r|R)\w+]eact [(j|J)\w+][(s|S)\w+]?)|(React Native?)|(Node JS?)|([(g|G)\w+]it?)|(Visual Studio?)|(Visual Studio Code?)|(Django?)|(Flask?)|(API REST?)|(http?s?)|(HTTP?S?)')
        
        
    def get_niv_etudes_re(self):
        #Niveau d'études requis : regex true
        return self._get_items('([(f|F)\w+]ormation?\s?[(b|B)\w+]ac\s\+\s[1-8])|([(b|B)\w+]ac\s\+\s[1-8])')
        
    def get_type_cursus_re(self):
        #Type de cursus (école ingénieur, master, autodidacte...) : regex true
        return self._get_items('([(é|E)\w+]cole [(i|I)\w+]ngénieur?)|([(m|M)\w+]aster?\s?\w{3,25})|([(a|A)\w+]utodidacte?)|([(g|G)\w+]rande[s]? [(é|E)\w+]cole[s]?)|([(é|E)\w+]cole[s]? de [(c|C)\w+]ommerce[s]?)|([(i|I)\w+]ngénieur [(i|I)\w+]nformatique?)')
        
    def get_type_contrat_re(self):
        #Type de contrat : regex true
        return self._get_items('[(c|C)\w+]ontrat?:?\s\w{3,25}')
    
    def get_categorie_developpeur_re(self):
        #Grande catégorie : développeur (developeur web, dev mobile,dev) / data regex true
        return self._get_items('([(d|D)\w+]éveloppeu[(r|s)]e? [(f|F)\w+]ull? [(s|S)\w+]tack)|([(d|D)\w+]éveloppeu[(r|s)]e? [(f|F)\w+]ront[(\s|\-)][(e|E)\w+]nd?)|(développeu[(r|s)]e? Back[(\s|\-)][(e|E)\w+]nd?)|([(d|D)\w+]éveloppeu[(r|s)]e? [(m|M)\w+]obile?)|([(d|D)\w+]éveloppeu[(r|s)]e? [(l|L)\w+]ogiciel?)|([(s|S)\w+]oftware [(i|I)\w+]ngenieer?)|([(d|D)\w+]éveloppeu[(r|s)]e? [(l|L)\w+]ogiciels?)|([(d|D)\w+]éveloppeu[(r|s)]e? [(C|c)\w+]\+\+?)|([(d|D)\w+]éveloppeu[(r|s)]e? [(c|C)\w+]#?)|([(d|D)\w+]éveloppeu[(r|s)]e? [(j|J)\w+]ava?)')
    
    def get_categorie_data_re(self):
        #Grande catégorie : data regex true
        return self._get_items('([(d|D)\w+]ata [(a|A)\w+]nalyst?)|([(d|D)\w+]ata [(s|S)\w+]cientist?)|([(i|I)\w+]ngénieu[(r|s)]e? [(m|M)\w+]achine [(i|I)\w+]earning?)|([(c|C)\w+]onsultant [(b|B)\w+]usiness [(i|I)\w+]ntelligence)|([(c|C)\w+]onsultant [(d|D)\w+]ata)|([(c|C)\w+]onsultant [(b|B)\w+]ig [(d|D)\w+]ata)|([(c|C)\w+]onsultant BI)')
        

In [10]:
dataset = pd.read_csv("indeed.csv")
print(len(dataset))
#for index, row in dataset.iterrows():
    #print(row["URL"])
    #indedRegexParser = IndedRegexParser(row["description"])
    #print(indedRegexParser._get_niveau_experience())
    #print(indedRegexParser._get_salaires())
    #print(indedRegexParser.get_outils_re())
    #print(indedRegexParser.get_niv_etudes_re())
    #print(indedRegexParser.get_type_cursus_re())
    #print(indedRegexParser.get_type_contrat_re())
    #print(indedRegexParser.get_categorie_developpeur_re())
    #print(indedRegexParser.get_categorie_data_re())


3632


In [248]:
dataset = pd.read_csv("indeed.csv")
dataset.columns

Index(['URL', 'Titre', 'Nom entreprise', 'Adresse', 'Date de publication',
       'salaire', 'description'],
      dtype='object')

In [215]:
import os
from orderedset import OrderedSet
class LocalIndeedPaser:
    def __init__(self):
        self.pages_path = 'C:\\Users\\User\\Documents\\Projet_ML_gr3\\scrapping\pages'
        self.dataset = pd.read_csv("indeed.csv")
        self.salary_pattern = "[[S|s]alaire?[\s+]?:?[\s+]?(.*)e?[\s+]?\/(an|mois)|((.*)?[\s+]?par?[\s+]?(an|ans|mois|jour|heure))"
    
    def _get_salary(self,select_result):
        salary = ""
        for item in select_result:
            if "€" in item.text:
                outer_salary = re.compile(self.salary_pattern)
                m_salary = outer_salary.search(item.text)
                if m_salary is not None:
                    salary = m_salary.group(0)
                    break
        return salary
    
    def save_file(self):
        self.dataset.to_csv("indeed.csv",index=False)
    
    def _get_listed_data(self, input_list):
        data = []
        for i in range(len(self.dataset)):
            inside_data = []
            for ele in input_list:
                pattern = re.compile(r"[\s/\(\),]"+ele+r"[\s/\(\),]")
                value = pattern.search(self.dataset['description'][i].lower().replace('\n',' ').replace('\r',' '))
                if value:
                    inside_data.append(1)
                else:
                    inside_data.append(0)
            data.append(inside_data)
        
        return data
    
    
    def parse_langage(self):
        languages = ['python', 'r','vba', 'mysql','excel','asp.net','nosql','sql','linux','mongodb',
            'mariadb','java','javascript','php','html','css','sas','c#','ruby','swift','objective-c',
            'vb.net','kotlin','scala','bash','powershell','shell','front end','back end','soap']
        
        data = self._get_listed_data(languages)
        
        language_dict = pd.DataFrame(data, columns=languages)
        self.language_df = pd.DataFrame.from_dict(language_dict)
        diff_cols = list(OrderedSet(self.dataset.columns) - OrderedSet(self.language_df.columns))
        self.dataset = self.dataset[diff_cols]
        self.dataset = pd.concat([self.dataset,self.language_df], axis=1)
    
    def parse_tools(self):
        tools = ['tableau', 'powerbi','symfony', 'jquery','angular','react','react native','node js','git','github',
            'visual studio','django','flask','api rest','laravel', 'hololens', 'docker', 'jira', 'scrum', 'kanban']
        
        data = self._get_listed_data(tools)

        tools_dict = pd.DataFrame(data, columns=tools)
        self.tools_df = pd.DataFrame.from_dict(tools_dict)
        diff_cols = list(OrderedSet(self.dataset.columns) - OrderedSet(self.tools_df.columns))
        self.dataset = self.dataset[diff_cols]
        self.dataset = pd.concat([self.dataset,self.tools_df], axis=1)
        
    
    def parse_salary(self):
        index = 0
        for root, dirs, files in os.walk(os.path.abspath(self.pages_path)):
            for file in files:
                local_file = os.path.join(root, file)
                file = open(local_file, "r") 
                html = file.read() 
                outer = re.compile(r'.*\-\-\-\-\-\-\-\-\-\-\-\-\-\-\-\-')
                m = outer.search(html)
                if m is not None:
                    link = m.group(0).replace("----------------","")
                    dataset_index = self.dataset[self.dataset["URL"] == link].index.values.astype(int)
                    #print(self.dataset["salaire"][dataset_index].values[0])
                    salary = self.dataset["salaire"][dataset_index]
                    
                    if (len(salary.values) == 0):
                        continue 
                        
                    if (salary.values[0] != "-"):
                        continue
                        
                    html = html.replace(m.group(0),"")
                    soup = BeautifulSoup(html, "html.parser")
                    result = soup.select(".jobsearch-JobMetadataHeader-item")
                    if len(result) > 0:
                        self.dataset["salaire"][dataset_index] = self._get_salary(result)
                        continue
                        
                    result = soup.select(".jobsearch-JobMetadataHeader-itemWithIcon .jobsearch-JobMetadataHeader-iconLabel")
                    if len(result) > 0:
                        self.dataset["salaire"][dataset_index] = self._get_salary(result)
                        continue
                    
                    result = soup.select("#jobDescriptionText")
                    if len(result) > 0:
                        index = index + 1 
                        outer_salary = re.compile(self.salary_pattern)
                        m_salary = outer_salary.search(result[0].text)
                        if m_salary is not None:
                            salary = m_salary.group(0)
                            print(salary)
                            break
                    
                    
                    

In [216]:
localIndeedPaser = LocalIndeedPaser()
localIndeedPaser.parse_tools()
localIndeedPaser.parse_langage()

In [217]:
#localIndeedPaser.language_df
localIndeedPaser.dataset.columns

Index(['URL', 'Titre', 'Nom entreprise', 'Adresse', 'Date de publication',
       'salaire', 'description', 'tableau', 'powerbi', 'symfony', 'jquery',
       'angular', 'react', 'react native', 'node js', 'git', 'github',
       'visual studio', 'django', 'flask', 'api rest', 'laravel', 'hololens',
       'docker', 'jira', 'scrum', 'kanban', 'python', 'r', 'vba', 'mysql',
       'excel', 'asp.net', 'nosql', 'sql', 'linux', 'mongodb', 'mariadb',
       'java', 'javascript', 'php', 'html', 'css', 'sas', 'c#', 'ruby',
       'swift', 'objective-c', 'vb.net', 'kotlin', 'scala', 'bash',
       'powershell', 'shell', 'front end', 'back end', 'soap'],
      dtype='object')

In [195]:
pip install orderedset

  Stored in directory: C:\Users\Junior\AppData\Local\pip\Cache\wheels\68\1a\0a\084d78f38459b3111414732e471b0cfbdf05b1931550f60ada
Successfully built orderedset
Note: you may need to restart the kernel to use updated packages.


In [25]:
localIndeedPaser.save_file()

In [28]:
xx = pd.read_csv("indeed.csv")

print(xx.iloc[35]["URL"])
print(xx.iloc[45]["URL"])
print(xx.iloc[48]["URL"])

https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYlbfkN0C5cUW69gWMagU3J5K1A_k01Vx9ltZoYwD7pptgAh3vLuJrZcHRVo6De2caH3Ort2lE6CgiOQWz6MGH2-A6pq2qoY9j86OzbwXgzXOxKY4gtptWN6wrWGMKCdwnMunrmFUWa1b4qMFgqsGZOoPYfhryqAalPWSpHwiewOGGYjBZysbv21AToMUnZ4BkANtbjLnbNuJaAQQiOw0k_9gXp_SH7g1L0J7jteMnxSlaW5Hv8c-mtBfNjnse7KGbemagx3qEvT2tTmd0X-KGH1KGoFg-6y477TtpQldV0JxQSxZ4wFN9y5vylvcphJJwuKs835iGwYy38X1jGKujH_jKv6diE-KS59h-w6DjlQWlvTDJcK5Afx7VqKOJrfLK4IkczilvjJUfCH2d3ATWrrJeSNquKTMXarhNP9DCiJ0GEOnYyBDaZ3hcPUhezK62SYTYho2M8eGnuthj6g==&p=4&fvj=1&vjs=3
https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYlbfkN0C5cUW69gWMagU3J5K1A_k01Vx9ltZoYwD7pptgAh3vLuJrZcHRVo6De2caH3Ort2lE6CgiOQWz6MGH2-A6pq2qoY9j86OzbwXgzXOxKY4gtptWN6wrWGMKCdwnMunrmFUWa1b4qMFgqsGZOoPYfhryV53Jsbjl8UxGrhbYwUgHnnWMJv5rJ6lhnCu9m8rGa0NYMr8D73KOXz0nu6d0_ux8153VSBl8r23TKnsadeKTn29e2rBgAlLq3SBd-UTPd_w7kE7pyR1PwtMKguKDTmwJDXeFCamNxpL1zkwNTojC0XLaUSWyFT8OcBDc-MWw7nSCHbVZSUswN8IwOhTilh_dKYnTC8tlGdGJPW3xuxNDHbdX_tBnV_DZPZ01suUxUlZSXULHIqNDGyzIyrF-emR-84p9XqqrTrwoh

In [431]:
xx

,URL,Titre,Nom entreprise,Adresse,Date de publication,salaire,description
0,https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYl...,Sup3r Développeur Web Full Stack @Startup,GlobeSailor SAS,Paris (75),2019-10-03 03:11:00.197236,NaN,4pp31 4 T0u5 135 D3v310p3r5 w3b full st4k !\nP...
1,https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYl...,Développeur Informatique H/F,VALENTE SECURYSTAR,94320 Thiais,2019-10-02 03:11:14.353355,1 850 € par mois,"VALENTE SECURYSTAR, fabricant de portes blindé..."
2,https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYl...,Développeur Android (H/F),Universign,Paris (75),2019-09-04 03:11:27.090785,NaN,Universign est le pure player de référence sur...
3,https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYl...,Développeur Ionic / Firebase,NEXTT,92200 Neuilly-sur-Seine,2019-09-19 03:11:39.836685,NaN,Dans le cadre du développement d'une applicati...
4,https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYl...,Data Engineer / Développeur NLP H/F,Wolters Kluwer,Paris (75),2019-09-04 03:11:53.014114,NaN,"Au sein du Pôle Droit et Réglementation, ratta..."
5,https://www.indeed.fr/pagead/clk?mo=r&ad=-6NYl...,Développeur Java H/F,Ratp Smart Systems,Paris (75),2019-09-10 03:12:05.948019,NaN,1er jet :\nPour accélérer sa transformation et...
6,https://www.indeed.fr/rc/clk?jk=6986aa9ce3ca25...,Développeur iOS (H/F),Universign,Paris (75),2019-10-01 03:12:18.742210,-,Universign est le pure player de référence sur...
7,https://www.indeed.fr/rc/clk?jk=8ee618fc0da425...,BALENCIAGA - Coordinateur/trice Développement ...,Groupe Kering,Paris (75),2019-10-03 16:12:31.688147,-,Summary\nFondée en 1917 par l’Espagnol Cristób...
8,https://www.indeed.fr/rc/clk?jk=2f40c0f1d86b7b...,Développeur Junior (ReactJS / Node.js),AKOYA,Paris (75),2019-10-01 03:12:44.296664,NaN,AKOYA GENIUS\nEn complément de ses activités d...
9,https://www.indeed.fr/rc/clk?jk=fdb86ccd993a5c...,BALENCIAGA - Coordinateur/trice Développement ...,Balenciaga,Paris (75),2019-10-03 15:12:57.009145,-,Summary\nFondée en 1917 par l’Espagnol Cristób...
